In [1]:
import pandas as pd
pd.options.display.max_rows=6

import numpy as np
import datetime
import sys

# match tables

In [4]:
sites = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_sites_20170909_20170910_032552_607997080.csv.gz')
activity_cats = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_activity_cats_20170909_20170910_033227_607997440.csv.gz')
#paid_search = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_paid_search_20170909_20170910_032551_607997437.csv.gz')
#ads = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/match_tables/dcm_account7252_match_table_ads_20170712_20170713_034624_583169409.csv')

### paid_search

In [40]:
import os
ps_dir = '/home/jubauser1/jzou/dcm_account7252/match_tables/paid_search/'
_ps = os.listdir(ps_dir)
paid_search = pd.DataFrame()
for _ in _ps:
    paid_search = paid_search.append(pd.read_csv(ps_dir+_, dtype=str), ignore_index=True)

In [41]:
search_click = pd.read_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/cost_outer_source/callaway_cost_by_kw_by_date.csv', 
    dtype={'keywordId': str})

search_click['keywordMatchType'] = search_click['keywordMatchType'].apply(
    lambda x: x.upper() if type(x)==str else x)

search_click['cpc'] = search_click['cost']/search_click['clicks']

In [42]:
kws_in_ds = search_click[['keywordId', 'campaign', 'adGroup', 'keywordMatchType']].drop_duplicates()

In [43]:
paid_search = paid_search.merge(
    kws_in_ds, 
    left_on=['Paid Search Keyword ID', 'Paid Search Campaign', 'Paid Search Ad Group', 'Paid Search Match Type'], 
    right_on=['keywordId', 'campaign', 'adGroup', 'keywordMatchType'],
    how='left')

In [44]:
paid_search = paid_search[paid_search['keywordId'].notnull()]

paid_search.drop_duplicates('Paid Search Legacy Keyword ID', keep='last', inplace=True)

# click

In [45]:
columns_click = ['Event Time', 'User ID', 'Site ID (DCM)', 'Segment Value 1']
dtype_click = {'Event Time': int, 'User ID': str, 'Site ID (DCM)': int}
click = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/click_callaway_5_campaigns.csv',
                    usecols=columns_click, dtype=dtype_click)

click = click[click['User ID'] != '0']
click = click[click['Site ID (DCM)'] != 4036006] # 'DART Search : Yahoo! Gemini'

click['date'] = click['Event Time'].apply(
    lambda x: datetime.datetime.fromtimestamp(x/1000000).strftime('%Y-%m-%d'))
click = click[click['date'] <= '2017-08-15']

click = click.merge(sites[['Site ID (DCM)', 'Site (DCM)']], on='Site ID (DCM)', how='left')

In [46]:
click['Segment Value 1'] = click['Segment Value 1'].apply(lambda x: x if np.isnan(x) else str(int(x)))

click = click.merge(paid_search[['Paid Search Legacy Keyword ID', 'Paid Search Keyword', 'Paid Search Keyword ID',
                                 'Paid Search Campaign', 'Paid Search Ad Group',
                                 'Paid Search Match Type'#, 'Paid Search Ad ID'
                                ]], 
                    left_on='Segment Value 1', right_on='Paid Search Legacy Keyword ID', how='left')

In [47]:
#click['Paid Search Keyword'] = click['Paid Search Keyword'].fillna('empty')
def brand_or_non(x, brand):
    if type(x) == str:
        if brand in x:
            return 'branded_'
        else:
            return 'nonbranded_'
    else:
        return ''
click['brand_or_non'] = click['Paid Search Keyword'].apply(lambda x: brand_or_non(x, 'callaway'))

click['activity_cat'] = click['brand_or_non']+'click: '+click['Site (DCM)']

### est_click_cost

In [48]:
search_cpc = click.merge(search_click, 
                        left_on=['Paid Search Campaign', 
                                 'Paid Search Ad Group', 
                                 'Paid Search Keyword ID', 'Paid Search Match Type',
                                 'date'], 
                        right_on=['campaign', 
                                  'adGroup', 'keywordId', 'keywordMatchType', 
                                  'date'],
                        how='left')

search_cpc = search_cpc['cpc']

search_cpc = search_cpc[search_cpc.notnull()]

In [19]:
search_cpc

52        0.847500
61        1.447500
64        0.847500
            ...   
141257    1.020000
141260    2.707500
141263    1.171538
Name: cpc, Length: 63080, dtype: float64

In [49]:
pla_cost = pd.read_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/cost_outer_source/Callaway Apparel - pla cost.csv',
    parse_dates=['From'])
pla_cost = pla_cost.groupby(['Campaign', 'From'])['Clicks', 'Cost'].sum()
pla_cost.reset_index(inplace=True)

pla_cost['Site (DCM)'] = pla_cost['Campaign'].apply(
    lambda x: {'B_Bing Shopping_EST-2773': 'DART Search : MSN',
               'G_Google Shopping_Callaway Apparel_EST-2773': 'DART Search : Google'}[x])

pla_cost['cpc'] = pla_cost['Cost'] / pla_cost['Clicks']

pla_cost['date'] = pla_cost['From'].apply(pd.to_datetime).apply(lambda x:x.strftime('%Y-%m-%d'))

pla_cpc = click.merge(pla_cost, on=['Site (DCM)', 'date'], how='left')

pla_cpc = pla_cpc[(pla_cpc['Site (DCM)'].isin({'DART Search : Google', 'DART Search : MSN'})) & 
                  (pla_cpc['Paid Search Keyword ID'].isnull())]['cpc']

In [50]:
pla_cpc

76             NaN
86             NaN
106            NaN
            ...   
141251    1.203333
141252    1.203333
141262    1.203333
Name: cpc, Length: 41389, dtype: float64

In [51]:
fb_click = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/cost_outer_source/PEI-Callaway-Mar-1-2017-_-Sep-13-2017.csv')

fb_click['cpc'] = fb_click['Amount Spent (USD)'] / fb_click['Clicks (All)']

fb_click['site'] = 'Facebook'

fb_cpc = click.merge(
    fb_click, left_on=['Site (DCM)', 'date'], right_on=['site', 'Reporting Starts'], how='left')

fb_cpc = fb_cpc[fb_cpc['Site (DCM)'] == 'Facebook']['cpc']

In [52]:
fb_cpc

6        NaN
7        NaN
8        NaN
          ..
124348   NaN
125683   NaN
138123   NaN
Name: cpc, Length: 14422, dtype: float64

In [53]:
cpc = pd.concat([fb_cpc, search_cpc, pla_cpc])

cpc

6              NaN
7              NaN
8              NaN
            ...   
141251    1.203333
141252    1.203333
141262    1.203333
Name: cpc, Length: 118891, dtype: float64

In [54]:
click = click.join(cpc)

In [64]:
fb_cpc.mean()

0.35272123932245797

In [68]:
click.set_value(click[(click['Site (DCM)']=='Facebook') & (click['date']<'2017-04-30')].index, 
                'cpc', 0.35272123932245797)

,Event Time,User ID,Site ID (DCM),Segment Value 1,date,Site (DCM),Paid Search Legacy Keyword ID,Paid Search Keyword,Paid Search Keyword ID,Paid Search Campaign,Paid Search Ad Group,Paid Search Match Type,brand_or_non,activity_cat,cpc
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,1695745,NaN,2017-03-30,QuantCast,NaN,NaN,NaN,NaN,NaN,NaN,,click: QuantCast,NaN
1,1490969647208748,AMsySZagAWzbVDPRBTEAXJN7YQ7s,1695745,NaN,2017-03-31,QuantCast,NaN,NaN,NaN,NaN,NaN,NaN,,click: QuantCast,NaN
2,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,1695745,NaN,2017-03-31,QuantCast,NaN,NaN,NaN,NaN,NaN,NaN,,click: QuantCast,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141262,1502855931488876,AMsySZabKw6hZJkJzLuQIJoS_kM2,1234535,19878876612,2017-08-15,DART Search : Google,NaN,NaN,NaN,NaN,NaN,NaN,,click: DART Search : Google,1.203333
141263,1502855243083630,AMsySZZ1movmk7onzi-ZtZwWtkrO,1234535,19839135300,2017-08-15,DART Search : Google,19839135300,+callaway +golf +shoes,43700019839135300,G_Brand_Callaway Apparel_ Shoes _ EST-2773,General - BMM,BROAD,branded_,branded_click: DART Search : Google,1.171538
141264,1502854557585647,AMsySZZnRXr2xO5oY4BUWfMTHaPV,3446099,NaN,2017-08-15,Criteo,NaN,NaN,NaN,NaN,NaN,NaN,,click: Criteo,NaN


In [70]:
click.to_csv('/home/jubauser1/jzou/dcm_account7252/results/click_callaway_5_campaigns_with_cost.csv', index=False)

In [5]:
click = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/click_callaway_5_campaigns_with_cost.csv')

# impression

In [3]:
columns_impr = ['Campaign ID', 'Event Time', 'User ID', 'Site ID (DCM)', 
                'DBM Media Cost (USD)', 'DBM Revenue (USD)',
                'DBM Total Media Cost (USD)', 'DBM Billable Cost (USD)']
dtype_impr = {'Campaign ID': int, 'Event Time': int, 'User ID': str, 'Site ID (DCM)': int,
              'DBM Media Cost (USD)': int, 'DBM Revenue (USD)': int,
              'DBM Total Media Cost (USD)': int, 'DBM Billable Cost (USD)': int}

impression = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/impression_callaway_5_campaigns_reduced.csv',
                         usecols=columns_impr, 
                         dtype=dtype_impr
                        )

In [4]:
impression = impression[impression['User ID'] != '0']

impression = impression.merge(sites[['Site ID (DCM)', 'Site (DCM)']], on='Site ID (DCM)', how='left')

impression['activity_cat'] = 'impr: '+impression['Site (DCM)']

impression['date'] = impression['Event Time'].apply(
    lambda x: datetime.datetime.fromtimestamp(x/1000000).strftime('%Y-%m-%d'))
impression = impression[impression['date'] <= '2017-08-15']

impression

,Campaign ID,DBM Billable Cost (USD),DBM Media Cost (USD),DBM Revenue (USD),DBM Total Media Cost (USD),Event Time,Site ID (DCM),User ID,Site (DCM),activity_cat,date
0,11222752,0,0,0,0,1490974787250049,1695745,AMsySZYCihuTC9doIqCbWK9sjhIm,QuantCast,impr: QuantCast,2017-03-31
1,11222752,0,0,0,0,1490974738277507,1695745,AMsySZZ-hB0xKXlxMDSs37YP7Q8o,QuantCast,impr: QuantCast,2017-03-31
2,11222752,0,0,0,0,1490973683656069,1695745,AMsySZY49kkJwdR8ITeGasKc5uiQ,QuantCast,impr: QuantCast,2017-03-31
...,...,...,...,...,...,...,...,...,...,...,...
8823205,20073454,0,0,0,0,1502855373165195,1695745,AMsySZaBEqGWjARG8ExYr3168HpH,QuantCast,impr: QuantCast,2017-08-15
8823206,20073454,0,0,0,0,1502855907646410,1695745,AMsySZZf8V8S9ehxOMnhI3pKQpJ8,QuantCast,impr: QuantCast,2017-08-15
8823207,20073454,0,0,0,0,1502853679008068,1695745,AMsySZbYTbW6NSpGqf7dTejWdQlH,QuantCast,impr: QuantCast,2017-08-15


### est_impr_cost

In [5]:
impr_cost = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/cost_outer_source/Copy of Callaway Costs Daily.csv')

In [6]:
impr_cost['date'] = impr_cost['Date'].apply(lambda x: datetime.datetime.strptime(x,'%m/%d/%Y').strftime('%Y-%m-%d'))

In [28]:
impr_cost.columns

Index(['Date', 'Rakuten Marketing Index', 'Criteo', 'QuantCast', 'date'], dtype='object')

In [24]:
impr_cost.rename(columns={'Rakuten Spend': 'Rakuten Marketing Index', 
                          'Criteo Spend ': 'Criteo', 
                          'Quancast Spend ': 'QuantCast'},
                inplace=True)

In [29]:
l = list()
for i in impr_cost.index:
    date = impr_cost.loc[i, 'date']
    for s in ['Rakuten Marketing Index', 'Criteo', 'QuantCast']:
        d = {
            'Site (DCM)': s,
            'date': date,
            'cost': impr_cost.loc[i, s]
        }
        l.append(d)

In [31]:
pd.DataFrame(l)

,Site (DCM),cost,date
0,Rakuten Marketing Index,0.00,2017-04-03
1,Criteo,52.77,2017-04-03
2,QuantCast,56.00,2017-04-03
...,...,...,...
501,Rakuten Marketing Index,0.00,2017-09-17
502,Criteo,0.00,2017-09-17
503,QuantCast,60.35,2017-09-17


In [13]:
impr_by_site_by_date = impression.groupby(['Site (DCM)', 'date']).apply(len)

impr_by_site_by_date.name = 'impressions'

impr_by_site_by_date = impr_by_site_by_date.reset_index()

In [34]:
impr_by_site_by_date = impr_by_site_by_date.merge(pd.DataFrame(l), on=['Site (DCM)', 'date'], how='left')

In [35]:
impr_by_site_by_date['cpm'] = impr_by_site_by_date['cost']/impr_by_site_by_date['impressions']

In [36]:
impr_by_site_by_date

,Site (DCM),date,impressions,cost,cpm
0,Criteo,2017-04-03,14582,52.77,0.003619
1,Criteo,2017-04-04,255,0.33,0.001294
2,Criteo,2017-04-05,4,0.00,0.000000
...,...,...,...,...,...
260,QuantCast,2017-08-13,77447,295.28,0.003813
261,QuantCast,2017-08-14,78281,298.56,0.003814
262,QuantCast,2017-08-15,73770,280.87,0.003807


In [38]:
impression = impression.merge(impr_by_site_by_date, on=['Site (DCM)', 'date'], how='left')

In [39]:
impression.to_csv('/home/jubauser1/jzou/dcm_account7252/results/impression_callaway_5_campaigns_reduced_with_cost.csv', 
                  index=False)

In [6]:
impression = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/impression_callaway_5_campaigns_reduced_with_cost.csv')

# activity

In [7]:
columns_activity = ['Event Time', 'User ID', 'Site ID (DCM)', 'Activity ID', 
                    'Total Conversions', 'Total Revenue']
dtype_activity = {'Event Time': int, 'User ID': str, 'Site ID (DCM)': int, 'Activity ID': int, 
                  'Total Conversions': int}

activity = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/results/activity_callaway_5_campaigns.csv',
                       usecols=columns_activity, dtype=dtype_activity)

activity = activity[activity['User ID'] != '0']

In [8]:
#activity = activity.merge(ads[['Ad ID', 'Ad']], on='Ad ID', how='left')
activity = activity.merge(sites[['Site ID (DCM)', 'Site (DCM)']], on='Site ID (DCM)', how='left')
activity = activity.merge(activity_cats[['Activity ID', 'Activity']], on='Activity ID', how='left')
activity = activity[activity['Activity'].str.contains('Callaway')]

In [9]:
hva1 = ['Callaway - About Us (Who We Are)',
        'Callaway - Create Account',
        'Callaway - Subscription (Join Our Email List)',
        'Callaway - Subscription Success',
        'Callaway - Log In (My Account)',
        'Callaway - Shipping & Handling',
        'Callaway - Shipping - All',
        'Callaway - Shipping - Guest',
        'Callaway - Shipping - Registered',]

hva2 = ['Callaway - Big & Tall',
        'Callaway - Clearance',
        'Callaway - Unsubscribe - HVA 2',
        'Callaway - Search - HVA2',
        'Callaway - Big & Tall_Belts',
        'Callaway - Big & Tall_Outerwear',
        'Callaway - Big & Tall_Pants-Shorts',
        'Callaway - Big & Tall_Polos',
        'Callaway - Clearance_Big & Tall Clearance',
        'Callaway - Clearance_Mens Clearance',
        'Callaway - Clearance_Womens Clearance',
        'Callaway - Features_Best-Sellers',
        'Callaway - Features_Callaway X',
        'Callaway - Features_New Arrivals',
        'Callaway - Features_Opti-Series',
        'Callaway - Men Golf Shoes',
        'Callaway - Men_Belts',
        'Callaway - Men_Outerwear',
        'Callaway - Men_Pants',
        'Callaway - Men_Polos',
        'Callaway - Men_Shorts',
        'Callaway - Men_Standard Collection',
        'Callaway - Products - Belts',
        'Callaway - Products - Jacket',
        'Callaway - Products - Jackets',
        'Callaway - Products - Men Golf Shoes',
        'Callaway - Products - Pants',
        'Callaway - Products - Polo',
        'Callaway - Products - Polos',
        'Callaway - Products - Short',
        'Callaway - Products - Shorts',
        'Callaway - Products - Skorts',
        'Callaway - Products - Sweaters',
        'Callaway - Products - Vests',
        'Callaway - Products - Women Golf Shoes',
        'Callaway - Women Golf Shoes',
        'Callaway - Women_Belts',
        'Callaway - Women_Outerwear',
        'Callaway - Women_Pants',
        'Callaway - Women_Polos',
        'Callaway - Women_Skorts & Shorts',
        'Callaway - Women_Standard Collection',
        'Callaway  - Products - Universal',
        'Callaway - About Us (Who We Are)',
        'Callaway - Accessibility Statement',
        'Callaway - Accessories_Belts',
        'Callaway - Accessories_Hats',
        'Callaway - Accessories_Sunglasses',
        'Callaway - Women']

hva3 = ['Callaway - Add to Cart - HVA 3',
        'Callaway - Order Status (Orders)',
        'Callaway - Billing_Payment',
        'Callaway - Check Out',
        'Callaway - Order Review',
        'Callaway - Paypal',
        #'Callaway - Secure Checkout',
        'Callaway - Shipping',
        'Callaway - Billing_Payment - All',
        'Callaway - Billing_Payment - Guest',
        'Callaway - Billing_Payment - Registered',
        'Do Not Use - Callaway - Secure Checkout']

transaction = [#'Callaway - Thank You Page Items Sold', 
               'Callaway - Thank You Page Transactions']

activity_to_cat = dict()
for x in hva1:
    activity_to_cat[x] = 'HVA1'
    
for x in hva2:
    activity_to_cat[x] = 'HVA2'

for x in hva3:
    activity_to_cat[x] = 'HVA3'
    
for x in transaction:
    activity_to_cat[x] = 'transaction'

activity['activity_cat'] = activity['Activity'].apply(lambda x : activity_to_cat.get(x, 'others'))

activity['total_hva'] = activity.apply(lambda row: row['Total Conversions'] if row['activity_cat'] in {'HVA1', 'HVA2', 'HVA3'} else 0, axis=1)

# groups

In [10]:
buyers = set(activity[(activity['activity_cat'] == 'transaction') & 
                      (activity['Total Conversions'] > 0)]['User ID'])
len(buyers)

4736

In [11]:
groups_by_id = activity[~activity['User ID'].isin(buyers)].groupby('User ID')

hva_by_id = groups_by_id['total_hva'].sum()

average_hva = hva_by_id.mean()
average_hva

5.7742297071930446

In [12]:
high_engagers = set(hva_by_id[hva_by_id >= average_hva].index)
len(high_engagers)

26278

In [13]:
low_engagers = set(hva_by_id[(hva_by_id < average_hva)
                             #& (hva_by_id > 0)
                            ].index)
len(low_engagers)

83453

# impression and click by group

In [14]:
impression['cost'] = impression['cpm']
click['cost'] = click['cpc']
activity['cost'] = None
impression['revenue'] = None
click['revenue'] = None
activity['revenue'] = activity['Total Revenue'] * 1000000

In [15]:
activity = activity[activity['activity_cat']!='others']

In [16]:
merged_columns = ['Event Time', 'User ID', 'activity_cat', 'cost', 'revenue']
merged = impression[merged_columns].append(click[merged_columns], ignore_index=True)
merged = merged.append(activity[merged_columns], ignore_index=True)
merged.sort_values(by='Event Time', inplace=True)

merged.reset_index(inplace=True, drop=True)

In [17]:
merged

,Event Time,User ID,activity_cat,cost,revenue
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,click: QuantCast,NaN,None
1,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,click: QuantCast,NaN,None
2,1490969529196517,AMsySZbCkYLEzuC2-R0IJyWJh51l,click: QuantCast,NaN,None
...,...,...,...,...,...
9878709,1504497596191373,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0
9878710,1504497596212905,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0
9878711,1504497599912465,AMsySZaAToWbFircoFIR0y8Ipxhd,HVA2,NaN,0


In [18]:
merged['group'] = 4
merged.set_value(merged[merged['User ID'].isin(buyers)].index, 'group', 1)
merged.set_value(merged[merged['User ID'].isin(high_engagers)].index, 'group', 2)
merged.set_value(merged[merged['User ID'].isin(low_engagers)].index, 'group', 3)

,Event Time,User ID,activity_cat,cost,revenue,group
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,click: QuantCast,NaN,None,3
1,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,click: QuantCast,NaN,None,4
2,1490969529196517,AMsySZbCkYLEzuC2-R0IJyWJh51l,click: QuantCast,NaN,None,4
...,...,...,...,...,...,...
9878709,1504497596191373,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0,3
9878710,1504497596212905,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0,3
9878711,1504497599912465,AMsySZaAToWbFircoFIR0y8Ipxhd,HVA2,NaN,0,3


In [19]:
merged.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/merged_with_cost_and_group_no_others_2017-08-15.csv', index=False)

In [111]:
buyer_act = merged[merged['User ID'].isin(buyers)]

In [112]:
buyer_1st_transaction = pd.DataFrame()
for user_id, act_group in buyer_act.groupby('User ID'):
    index_of_1st_transaction = act_group.index[act_group['activity_cat'] == 'transaction'][0]
    buyer_1st_transaction = buyer_1st_transaction.append(act_group.loc[:index_of_1st_transaction])

In [88]:
freq_buyer = buyer_1st_transaction['activity_cat'].value_counts()
freq_buyer.name = 'buyer'

In [89]:
high_engager_act = merged[merged['User ID'].isin(high_engagers)]

freq_high_engager = high_engager_act['activity_cat'].value_counts()
freq_high_engager.name = 'high_engager'

In [90]:
low_engager_act = merged[merged['User ID'].isin(low_engagers)]

freq_low_engager = low_engager_act['activity_cat'].value_counts()
freq_low_engager.name = 'low_engager'

In [91]:
no_visit_act = merged[~merged['User ID'].isin(high_engagers.union(low_engagers).union(buyers))]

freq_no_visit = no_visit_act['activity_cat'].value_counts()
freq_no_visit.name = 'no_visit'

In [125]:
len(set(no_visit_act['User ID']))

2262496

In [92]:
freq_by_group = pd.DataFrame(freq_buyer).join(
    freq_high_engager, how='outer').join(
    freq_low_engager, how='outer').join(
    freq_no_visit, how='outer')

In [93]:
freq_by_group.sort_index().to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/frequency_by_group.csv')

In [94]:
freq_by_group

,buyer,high_engager,low_engager,no_visit
HVA1,13733,11298.0,2461.0,NaN
HVA2,117393,452410.0,128636.0,NaN
HVA3,54435,37571.0,1236.0,NaN
...,...,...,...,...
nonbranded_click: DART Search : MSN,59,365.0,1742.0,534.0
others,168582,517641.0,270464.0,NaN
transaction,4736,NaN,NaN,NaN


In [133]:
buyer_1st_transaction['cost'].sum(), buyer_1st_transaction[buyer_1st_transaction['activity_cat'] == 'transaction']['revenue'].sum()

(3919.9633583497384, 588939.9199999988)

In [122]:
merged.groupby('group')['cost'].sum()

group
1     5727.569112
2    27178.650553
3    77902.292484
4    24948.496932
Name: cost, dtype: float64

In [123]:
merged.groupby('group')['revenue'].sum()

group
1    1373314.49
2          0.00
3          0.00
4           NaN
Name: revenue, dtype: float64

In [132]:
buyer_act[buyer_act['revenue'] > 0]

,Event Time,User ID,activity_cat,cost,revenue,group
6101,1491132099762598,AMsySZYw0CKfR8LS1hLJ3VzhCHNQ,others,NaN,144.94,1
6102,1491132099762936,AMsySZYw0CKfR8LS1hLJ3VzhCHNQ,transaction,NaN,144.94,1
13323,1491143891786039,AMsySZYYR0w6tELpcfXcRlw5dM9U,transaction,NaN,84.49,1
...,...,...,...,...,...,...
10920562,1504495424345887,AMsySZbPwVfI73R6oB4IS97kaTFi,transaction,NaN,112.17,1
10920769,1504497592948737,AMsySZZlT9EX-HM8-mE7VC7YIWSp,others,NaN,68.41,1
10920770,1504497592949476,AMsySZZlT9EX-HM8-mE7VC7YIWSp,transaction,NaN,68.41,1


In [131]:
buyer_act[buyer_act['User ID'] == 'AMsySZZlT9EX-HM8-mE7VC7YIWSp']

,Event Time,User ID,activity_cat,cost,revenue,group
42147,1491175400643243,AMsySZZlT9EX-HM8-mE7VC7YIWSp,impr: QuantCast,NaN,None,1
183376,1491368022284708,AMsySZZlT9EX-HM8-mE7VC7YIWSp,impr: QuantCast,4.194455e-07,None,1
240240,1491453297696814,AMsySZZlT9EX-HM8-mE7VC7YIWSp,impr: QuantCast,2.020200e-03,None,1
...,...,...,...,...,...,...
10920766,1504497559892157,AMsySZZlT9EX-HM8-mE7VC7YIWSp,others,NaN,0,1
10920769,1504497592948737,AMsySZZlT9EX-HM8-mE7VC7YIWSp,others,NaN,68.41,1
10920770,1504497592949476,AMsySZZlT9EX-HM8-mE7VC7YIWSp,transaction,NaN,68.41,1


# time span by group

In [126]:
def timestamp(x):  
    return datetime.datetime.fromtimestamp(x/1000000)

In [137]:
buyer_time_span = pd.DataFrame()
for user_id, act_group in buyer_1st_transaction.groupby('User ID'):
    d = {'user_id': user_id}
    try:
        index_of_1st_hva = act_group.index[act_group['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})][0]
    except IndexError:
        index_of_1st_hva = None
        
    try:
        index_of_1st_exposure = act_group.index[act_group['activity_cat'].isin({
             'branded_click: DART Search : Google',
             'branded_click: DART Search : MSN',
             'click: Criteo',
             'click: DART Search : Google',
             'click: DART Search : MSN',
             'click: Facebook',
             'click: QuantCast',
             'impr: Criteo',
             'impr: QuantCast',
             'nonbranded_click: DART Search : Google',
             'nonbranded_click: DART Search : MSN',})][0]
    except IndexError:
        index_of_1st_exposure = None
        
    try: 
        index_of_1st_transaction = act_group.index[act_group['activity_cat'] == 'transaction'][0]   
    except IndexError:
        index_of_1st_transaction = None
    
    if index_of_1st_transaction and index_of_1st_hva:
        d['1st_HVA_to_1st_transaction'] = (timestamp(act_group.loc[index_of_1st_transaction, 'Event Time']) - \
            timestamp(act_group.loc[index_of_1st_hva, 'Event Time'])).total_seconds()
    else:
        d['1st_HVA_to_1st_transaction'] = None
    
    if index_of_1st_transaction and index_of_1st_exposure:
        d['1st_exposure_to_1st_transaction'] = (timestamp(act_group.loc[index_of_1st_transaction, 'Event Time']) - \
            timestamp(act_group.loc[index_of_1st_exposure, 'Event Time'])).total_seconds()
    else:
        d['1st_exposure_to_1st_transaction'] = None
        
    if index_of_1st_hva and index_of_1st_exposure:
        d['1st_exposure_to_1st_HVA'] = (timestamp(act_group.loc[index_of_1st_hva, 'Event Time']) - \
            timestamp(act_group.loc[index_of_1st_exposure, 'Event Time'])).total_seconds()
    else:
        d['1st_exposure_to_1st_HVA'] = None
    
    buyer_time_span = buyer_time_span.append(d, ignore_index=True)

In [138]:
buyer_time_span

,1st_HVA_to_1st_transaction,1st_exposure_to_1st_HVA,1st_exposure_to_1st_transaction,user_id
0,752.324,3.37457,7.556984e+02,AMsySZY-3FnuknZAk_Iuw_o7vZXF
1,1.65241e+06,31.9301,1.652438e+06,AMsySZY-3texhhoZd8Dov2qqQvpN
2,91124.4,4.44869e+06,4.539812e+06,AMsySZY-BLK48LeaEQ2Na54gg8xM
...,...,...,...,...
4733,9.15434e+06,769356,9.923691e+06,AMsySZbzvNKPEjY24T4ae9YRu1u9
4734,211.599,53.4679,2.650665e+02,AMsySZbzvjSvjfHe5hdYH9wNZYxP
4735,1.60145e+06,4.24589,1.601451e+06,AMsySZbzzXQ_uBd53piNXo4vEbDm


In [139]:
buyer_time_span['1st_HVA_to_1st_transaction (minute)'] = buyer_time_span['1st_HVA_to_1st_transaction']/60
buyer_time_span['1st_HVA_to_1st_transaction (hour)'] = buyer_time_span['1st_HVA_to_1st_transaction']/3600
buyer_time_span['1st_HVA_to_1st_transaction (day)'] = buyer_time_span['1st_HVA_to_1st_transaction']/3600/24

buyer_time_span['1st_exposure_to_1st_HVA (minute)'] = buyer_time_span['1st_exposure_to_1st_HVA']/60
buyer_time_span['1st_exposure_to_1st_HVA (hour)'] = buyer_time_span['1st_exposure_to_1st_HVA']/3600
buyer_time_span['1st_exposure_to_1st_HVA (day)'] = buyer_time_span['1st_exposure_to_1st_HVA']/3600/24

buyer_time_span['1st_exposure_to_1st_transaction (minute)'] = buyer_time_span['1st_exposure_to_1st_transaction']/60
buyer_time_span['1st_exposure_to_1st_transaction (hour)'] = buyer_time_span['1st_exposure_to_1st_transaction']/3600
buyer_time_span['1st_exposure_to_1st_transaction (day)'] = buyer_time_span['1st_exposure_to_1st_transaction']/3600/24

In [140]:
buyer_time_span.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/buyer_time_span.csv', index=False)

In [142]:
buyer_time_span['1st_HVA_to_1st_transaction'].mean()/3600/24, 'days'

(7.5349377765434555, 'days')

In [144]:
hva_to_transaction = list(buyer_time_span[buyer_time_span['1st_HVA_to_1st_transaction'].notnull()]['1st_HVA_to_1st_transaction'])

hva_to_transaction.sort()

In [145]:
hva_to_transaction[int(len(hva_to_transaction)*0.75)]/3600/24, 'days'

(2.0375388098842593, 'days')

In [146]:
hva_to_transaction[int(len(hva_to_transaction)*0.5)]/60, 'minutes'

(28.2049281, 'minutes')

In [147]:
hva_to_transaction[int(len(hva_to_transaction)*0.25)]/60, 'minutes'

(10.30299705, 'minutes')

In [148]:
buyer_time_span['1st_exposure_to_1st_HVA'].mean()/3600/24, 'days'

(7.9473627298549445, 'days')

In [149]:
exposure_to_hva = list(buyer_time_span[buyer_time_span['1st_exposure_to_1st_HVA'].notnull()]['1st_exposure_to_1st_HVA'])

exposure_to_hva.sort()

In [150]:
exposure_to_hva[int(len(exposure_to_hva)*0.75)]/3600/24, 'days'

(4.280740989849536, 'days')

In [151]:
exposure_to_hva[int(len(exposure_to_hva)*0.5)], 'seconds'

(53.075141, 'seconds')

In [152]:
exposure_to_hva[int(len(exposure_to_hva)*0.25)], 'seconds'

(9.479183, 'seconds')

In [153]:
buyer_time_span['1st_exposure_to_1st_transaction'].mean()/3600/24, 'days'

(15.851691729962853, 'days')

In [154]:
exposure_to_transaction = list(buyer_time_span[buyer_time_span['1st_exposure_to_1st_transaction'].notnull()]['1st_exposure_to_1st_transaction'])

exposure_to_transaction.sort()

In [155]:
exposure_to_transaction[int(len(exposure_to_transaction)*0.75)]/3600/24, 'days'

(16.972389584814817, 'days')

In [160]:
exposure_to_transaction[int(len(exposure_to_transaction)*0.5)]/3600, 'hours'

(10.845820556111111, 'hours')

In [161]:
exposure_to_transaction[int(len(exposure_to_transaction)*0.25)]/60, 'seconds'

(15.423910383333332, 'seconds')

In [163]:
high_engager_act = merged[merged['User ID'].isin(high_engagers)]

high_engager_1st_hva = pd.DataFrame()
for user_id, act_group in high_engager_act.groupby('User ID'):
    try:
        index_of_1st_hva = act_group.index[act_group['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})][0]
        high_engager_1st_hva = high_engager_1st_hva.append(act_group.loc[:index_of_1st_hva])
    except Exception as e:
        print(user_id)

In [164]:
high_engager_time_span = pd.DataFrame()
for user_id, act_group in high_engager_1st_hva.groupby('User ID'):
    d = {'user_id': user_id}
    try:
        index_of_1st_hva = act_group.index[act_group['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})][0]
        try:
            index_of_1st_exposure = act_group.index[act_group['activity_cat'].isin({
                 'branded_click: DART Search : Google',
                 'branded_click: DART Search : MSN',
                 'click: Criteo',
                 'click: DART Search : Google',
                 'click: DART Search : MSN',
                 'click: Facebook',
                 'click: QuantCast',
                 'impr: Criteo',
                 'impr: QuantCast',
                 'nonbranded_click: DART Search : Google',
                 'nonbranded_click: DART Search : MSN',})][0]
            d['1st_exposure_to_1st_HVA'] = (timestamp(act_group.loc[index_of_1st_hva, 'Event Time']) - \
                timestamp(act_group.loc[index_of_1st_exposure, 'Event Time'])).total_seconds()
        except IndexError:
            #print(user_id)
            d['1st_exposure_to_1st_HVA'] = None
        
    except IndexError:
        d['1st_exposure_to_1st_HVA'] = None
    
    high_engager_time_span = high_engager_time_span.append(d, ignore_index=True)

In [165]:
high_engager_time_span['1st_exposure_to_1st_HVA (minute)'] = high_engager_time_span['1st_exposure_to_1st_HVA']/60
high_engager_time_span['1st_exposure_to_1st_HVA (hour)'] = high_engager_time_span['1st_exposure_to_1st_HVA']/3600
high_engager_time_span['1st_exposure_to_1st_HVA (day)'] = high_engager_time_span['1st_exposure_to_1st_HVA']/3600/24

In [166]:
high_engager_time_span.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/high_engager_time_span.csv', index=False)

In [167]:
high_engager_time_span['1st_exposure_to_1st_HVA'].mean()/3600/24

7.899013704999013

In [168]:
exposure_to_hva = list(high_engager_time_span[high_engager_time_span['1st_exposure_to_1st_HVA'].notnull()]['1st_exposure_to_1st_HVA'])

exposure_to_hva.sort()

In [169]:
exposure_to_hva[int(len(exposure_to_hva)*0.75)]/3600

47.547844046388889

In [170]:
exposure_to_hva[int(len(exposure_to_hva)*0.5)]

27.569431000000002

In [171]:
exposure_to_hva[int(len(exposure_to_hva)*0.25)]

5.6886270000000003

In [172]:
low_engager_act = merged[merged['User ID'].isin(low_engagers)]

low_engager_1st_hva = pd.DataFrame()
for user_id, act_group in low_engager_act.groupby('User ID'):
    try:
        index_of_1st_hva = act_group.index[act_group['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})][0]
        low_engager_1st_hva = low_engager_1st_hva.append(act_group.loc[:index_of_1st_hva])
    except Exception as e:
        pass

In [175]:
low_engager_time_span = pd.DataFrame()
for user_id, act_group in low_engager_1st_hva.groupby('User ID'):
    d = {'user_id': user_id}
    try:
        index_of_1st_hva = act_group.index[act_group['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})][0]
        try:
            index_of_1st_exposure = act_group.index[act_group['activity_cat'].isin({
                 'branded_click: DART Search : Google',
                 'branded_click: DART Search : MSN',
                 'click: Criteo',
                 'click: DART Search : Google',
                 'click: DART Search : MSN',
                 'click: Facebook',
                 'click: QuantCast',
                 'impr: Criteo',
                 'impr: QuantCast',
                 'nonbranded_click: DART Search : Google',
                 'nonbranded_click: DART Search : MSN',})][0]
            d['1st_exposure_to_1st_HVA'] = (timestamp(act_group.loc[index_of_1st_hva, 'Event Time']) - \
                timestamp(act_group.loc[index_of_1st_exposure, 'Event Time'])).total_seconds()
        except IndexError:
            #print(user_id)
            d['1st_exposure_to_1st_HVA'] = None
        
    except IndexError:
        d['1st_exposure_to_1st_HVA'] = None
    
    low_engager_time_span = low_engager_time_span.append(d, ignore_index=True)

In [176]:
low_engager_time_span['1st_exposure_to_1st_HVA (minute)'] = low_engager_time_span['1st_exposure_to_1st_HVA']/60
low_engager_time_span['1st_exposure_to_1st_HVA (hour)'] = low_engager_time_span['1st_exposure_to_1st_HVA']/3600
low_engager_time_span['1st_exposure_to_1st_HVA (day)'] = low_engager_time_span['1st_exposure_to_1st_HVA']/3600/24

In [177]:
low_engager_time_span.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/low_engager_time_span.csv', index=False)

In [178]:
low_engager_time_span['1st_exposure_to_1st_HVA'].mean()/3600/24

4.1436344789875141

In [179]:
exposure_to_hva = list(low_engager_time_span[low_engager_time_span['1st_exposure_to_1st_HVA'].notnull()]['1st_exposure_to_1st_HVA'])

exposure_to_hva.sort()

In [180]:
exposure_to_hva[int(len(exposure_to_hva)*0.75)]

56.605944999999998

In [181]:
exposure_to_hva[int(len(exposure_to_hva)*0.5)]

12.832652

In [182]:
exposure_to_hva[int(len(exposure_to_hva)*0.25)]

4.1006869999999997

# media freq k-means

In [ ]:
merged

In [ ]:
merged_1st_transaction = merged[~merged['User ID'].isin(buyers)].append(buyer_1st_transaction, ignore_index=True)

In [ ]:
merged_1st_transaction

In [ ]:
l = list()
for user_id, group in merged_1st_transaction.groupby('User ID'):
    d = group['activity_cat'].value_counts()
    d['user_id'] = user_id
    l.append(d)

media_freq_by_user_id = pd.DataFrame(l)

In [ ]:
media_freq_by_user_id.fillna(0, inplace=True)

In [ ]:
media_freq_by_user_id

In [ ]:
media_freq_by_user_id['group'] = 4

In [ ]:
media_freq_by_user_id.set_value(media_freq_by_user_id['user_id'].isin(buyers), 'group', 1)

In [ ]:
media_freq_by_user_id.set_value(media_freq_by_user_id['user_id'].isin(high_engagers), 'group', 2)
media_freq_by_user_id.set_value(media_freq_by_user_id['user_id'].isin(low_engagers), 'group', 3)

In [ ]:
media_freq_by_user_id['group'].value_counts()

In [ ]:
media_freq_by_user_id.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/media_freq_by_user_id.csv', index=False)

## user path by group

In [ ]:
def path_list_to_str(path):
    '''
    Args: 
        path: list, ['a', 'a', 'b', 'c', 'c', 'c']
    Returns:
        str, 'a(2) -> b(1) -> c(3)'
    '''
    if 'transaction' in path:
        path = path[: path.index('transaction')+1]
    curr = path[0]
    count = 0
    result = list()
    for x in path:
        if x == 'others':
            continue
        elif x == curr:
            count += 1
        else:
            result.append('%s(%d)'%(curr, count))
            curr = x
            count = 1
    result.append('%s(%d)'%(curr, count))
    return ' -> '.join(result)

path_list_to_str(['a', 'a', 'others', 'b', 'others', 'c', 'c',  'c', 'others'])

In [ ]:
path = list(merged[merged['User ID'] == 'AMsySZbTx4mBNOHKocl_o0JhbYdj']['activity_cat'])

path_list_to_str(path)

In [ ]:
merged_groups_by_id = merged.groupby('User ID')

user_path_dict = dict()
for user, group in merged_groups_by_id:
    try:
        path_list = list(group['activity_cat'])
        #path_list = [x for x in path_list if x != 'others']
        path_list = [x for x in path_list if x not in {'others', 'HVA1', 'HVA2', 'HVA3'}]
        if len(path_list) > 0:
            path = path_list_to_str(path_list)
            logging.info('%s >>>> %s'%(user, path))
            user_path_dict[user] = path
    except Exception as e:
        print(user, e)

In [ ]:
user_path = pd.DataFrame({'user_id': list(user_path_dict.keys()), 'path': list(user_path_dict.values())})

In [ ]:
#user_path = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/user_path_added_branded_nonbranded.csv')

def user_group(user_id, buyers, high_engagers, low_engagers):
    if user_id in buyers:
        return 1
    elif user_id in high_engagers:
        return 2
    elif user_id in low_engagers:
        return 3
    else:
        return 4
    
user_path['group'] = user_path['user_id'].apply(lambda x: user_group(x, buyers, high_engagers, low_engagers))

user_path

In [ ]:
user_path.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/user_path_added_branded_nonbranded_no_hva.csv', index=False)

In [ ]:
user_path

In [ ]:
user_path.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/model/user_path.csv', index=False)

In [ ]:
buyer_path = user_path[user_path['user_id'].isin(buyers)]

In [ ]:
high_engagers_path = user_path[user_path['user_id'].isin(high_engagers)]

In [ ]:
low_engagers_path = user_path[user_path['user_id'].isin(low_engagers)]

In [ ]:
buyer_path_count = pd.DataFrame(buyer_path['path'].value_counts())

In [ ]:
high_engagers_path_count = pd.DataFrame(high_engagers_path['path'].value_counts())

In [ ]:
low_engagers_path_count = pd.DataFrame(low_engagers_path['path'].value_counts())

In [ ]:
writer = pd.ExcelWriter(path='/home/jubauser1/jzou/dcm_account7252/user_path/user_path_by_group_added_branded_nonbranded.xlsx')

In [ ]:
buyer_path.to_excel(writer, sheet_name='buyer_path')
buyer_path_count.to_excel(writer, sheet_name='buyer_path_count')
high_engagers_path.to_excel(writer, sheet_name='high_engagers_path')
high_engagers_path_count.to_excel(writer, sheet_name='high_engagers_path_count')
low_engagers_path.to_excel(writer, sheet_name='low_engagers_path')
low_engagers_path_count.to_excel(writer, sheet_name='low_engagers_path_count')

In [ ]:
writer.save()

# performance by group

In [ ]:
user_groups = {'buyers': buyers, 'high_engagers': high_engagers,'low_engagers': low_engagers}
groups_performance = pd.DataFrame()
for group_name, users in user_groups.items():
    print(group_name, len(users))
    group_performance = pd.Series({'group': group_name})

    group_impr = impression[impression['User ID'].isin(users)]
    group_performance = group_performance.append(group_impr['activity_cat'].value_counts())

    group_click = click[click['User ID'].isin(users)]
    group_performance = group_performance.append(group_click['activity_cat'].value_counts())

    group_activity = activity[activity['User ID'].isin(users)]
    group_performance = group_performance.append(group_activity['activity_cat'].value_counts())

    groups_performance = groups_performance.append(group_performance, ignore_index=True)

In [ ]:
groups_performance

In [ ]:
groups_performance = groups_performance[['group', 'impr: Criteo', 'impr: QuantCast', 
                     'click: Criteo', 'click: DART Search : Google',
                     'click: DART Search : MSN', 'click: DART Search : Yahoo! Gemini',
                                         'branded_click: DART Search : Google',
                     'branded_click: DART Search : MSN', 'branded_click: DART Search : Yahoo! Gemini',
                                         'nonbranded_click: DART Search : Google',
                     'nonbranded_click: DART Search : MSN', 'nonbranded_click: DART Search : Yahoo! Gemini',
                     'click: Facebook', 'click: QuantCast', 
                     'HVA1', 'HVA2', 'HVA3', 'others', 'transaction']]
groups_performance.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/groups_performance_added_branded_nonbranded.csv', index=False)

In [ ]:
merged_buyers = merged[merged['User ID'].isin(buyers)].copy()
merged_buyers.sort_values(by='Event Time', inplace=True)
merged_buyers.reset_index(inplace=True, drop=True)

merged_buyers

In [ ]:
time_span = list()
for user, df in merged_buyers.groupby('User ID'):
    index_of_1st_transaction = df[df['activity_cat'] == 'transaction'].index[0]

    df = df.loc[:index_of_1st_transaction]

    first_transaction = df.loc[index_of_1st_transaction]['Event Time']
    d = {'user_id': user} 
    
    temp = df[df['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3'})]
    if len(temp) > 0:
        first_hva = df.loc[temp.index[0]]['Event Time']
        d['1st HVA to 1st transaction'] = (timestamp(first_transaction) - timestamp(first_hva)).seconds
        index_of_1st_hva = temp.index[0]
    else:
        first_hva = None
        d['1st HVA to 1st transaction'] = None
    del temp

    temp = df[df['activity_cat'].isin({'HVA1', 'HVA2', 'HVA3', 'others'})]
    if len(temp) > 0:
        first_conversion = df.loc[temp.index[0]]['Event Time']
        d['1st conversion to 1st transaction'] = (timestamp(first_transaction) - timestamp(first_conversion)).seconds
    else:
        first_conversion = None
        d['1st conversion to 1st transaction'] = None
    del temp

    temp = df[df['activity_cat'].isin({'click: Criteo',
                                        'click: DART Search : Google',
                                        'click: DART Search : MSN',
                                        'click: DART Search : Yahoo! Gemini',
                                        'branded_click: DART Search : Google',
                                        'branded_click: DART Search : MSN',
                                        'branded_click: DART Search : Yahoo! Gemini',
                                        'nonbranded_click: DART Search : Google',
                                        'nonbranded_click: DART Search : MSN',
                                        'nonbranded_click: DART Search : Yahoo! Gemini',
                                        'click: Facebook',
                                        'click: QuantCast',
                                        'impr: Criteo',
                                        'impr: QuantCast'})]
    if len(temp) > 0:
        first_exposure = df.loc[temp.index[0]]['Event Time']
        last_exposure = df.loc[temp.index[-1]]['Event Time']
        d['1st exposure to 1st transaction'] = (timestamp(first_transaction) - timestamp(first_exposure)).seconds
        d['last exposure to 1st transaction'] = (timestamp(first_transaction) - timestamp(last_exposure)).seconds
        if first_hva:
            temp = temp[temp['Event Time'] < first_hva]
            if len(temp) > 0:
                first_exposure_before_hva = df.loc[temp.index[0]]['Event Time']
                last_exposure_before_hva = df.loc[temp.index[-1]]['Event Time']
                d['1st exposure to 1st hva'] = (timestamp(first_hva) - timestamp(first_exposure_before_hva)).seconds
                d['last exposure to 1st hva'] = (timestamp(first_hva) - timestamp(last_exposure_before_hva)).seconds
            else:
                d['1st exposure to 1st hva'] = None
                d['last exposure to 1st hva'] = None
        else:
            d['1st exposure to 1st hva'] = None
            d['last exposure to 1st hva'] = None
    else:
        d['1st exposure to 1st transaction'] = None
        d['last exposure to 1st transaction'] = None
    del temp
    time_span.append(d)

In [ ]:
time_span_df = pd.DataFrame(time_span)

In [ ]:
time_span_df.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/buyers_time_span_added_branded_nonbranded.csv', index=False)

In [ ]:
time_span_df

In [ ]:
def second_to_time(sec):
    days = sec // 86400
    sec -= 86400*days

    hrs = sec // 3600
    sec -= 3600*hrs

    mins = sec // 60
    sec -= 60*mins
    print(days, ':', hrs, ':', mins, ':', sec)
    
second_to_time(11823)

columns_mean = ['1st exposure to 1st transaction', 
                'last exposure to 1st transaction', 
                '1st conversion to 1st transaction',
                '1st HVA to 1st transaction',
                '1st exposure to 1st hva',
                'last exposure to 1st hva']
for c in columns_mean:
    print(c, '>>>>>',)
    second_to_time(time_span_df[time_span_df[c].notnull()][c].mean())

In [ ]:
merged_sample = merged[merged['User ID'].isin({
    'AMsySZYqZ62isOj4ct2oaN7cxzGm',
    'AMsySZbvt4-cUPcPkym8okp_IvkL',
    'AMsySZaecrQgZiLlYDllkIP-qrsI',
    'AMsySZZx2SL_OWuDkcb1kLFoxZSf',
    'AMsySZYPo16_Thv1pOtns8PChLud',
    'AMsySZanHUWC9p2FfsUw2bjrJK0u',
    'AMsySZYa2OIWa7yfPWOseKc2a8K0',
    'AMsySZaeArP_muso7MrpJOmZcBng',
    'AMsySZY3yrWNDMK_M7rgKIqdjBYS',
    'AMsySZa0hlZABPXiP6U2inxD3M-l'
    
})]

In [ ]:
merged_sample['date'] = merged_sample['Event Time'].apply(lambda x: timestamp(x).strftime('%Y-%m-%d'))
merged_sample['time'] = merged_sample['Event Time'].apply(lambda x: timestamp(x).strftime('%H:%M:%S'))

In [ ]:
merged_sample

In [ ]:
merged_sample.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/qc/sample_users.csv',index=False)

# group performance by user id

In [ ]:
merged

In [ ]:
groups_performace_by_id = pd.DataFrame(columns=['user_id', 'group', 'impr: Criteo', 'impr: QuantCast', 
                     'click: Criteo', 'click: DART Search : Google',
                     'click: DART Search : MSN', 'click: DART Search : Yahoo! Gemini',
                     'click: Facebook', 'click: QuantCast', 
                     'HVA1', 'HVA2', 'HVA3', 'others', 'transaction'])

In [ ]:
list_of_user = list()
for user, df in merged.groupby('User ID'):
    s = df['activity_cat'].value_counts()
    s['user_id'] = user
    if user in buyers:
        s['group'] = 'buyers'
    elif user in high_engagers:
        s['group'] = 'high_engagers'
    elif user in low_engagers:
        s['group'] = 'low_engagers'
    else:
        s['group'] = 'no_visit'
    logging.info(user)
    list_of_user.append(s)
    #groups_performace_by_id = groups_performace_by_id.append(s, ignore_index=True)

In [ ]:
groups_performace_by_id = pd.DataFrame(list_of_user)

In [ ]:
groups_performace_by_id.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/groups_performace_by_id.csv', index=False)

In [ ]:
groups_performace_by_id[['user_id', 'group', 'impr: Criteo', 'impr: QuantCast', 
                         'click: Criteo', 'click: DART Search : Google',
                         'click: DART Search : MSN', 'click: DART Search : Yahoo! Gemini',
                         'click: Facebook', 'click: QuantCast', 
                         'HVA1', 'HVA2', 'HVA3', 'others', 'transaction']].to_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/groups_performace_by_id.csv', index=False)

In [ ]:
groups_performace_by_id

# path permutation

In [ ]:
import json
import re
import sys

In [ ]:
def extract_activity_count(s):
    '''
    Args: str, eg. 'impr(23)'
    Returns:
    tuple, (str, int), eg. ('impr', 23)
    '''
    m = re.match('(.*)\((\d+)\)', s)
    return m.group(1), int(m.group(2))

extract_activity_count('click: DART Search : Yahoo! Gemini(23)')

In [ ]:
user_path = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/user_path_added_branded_nonbranded.csv')

In [ ]:
user_path

In [ ]:
def path_to_list(s):
    '''
    Args:
    str, path str, eg. 'impr(1) -> click(2) -> hva(3)'
    Returns:
    list of activity, eg.['impr', 'click', 'hva']
    '''
    l = s.split(' -> ')
    l = [extract_activity_count(x)[0] for x in l]
    return l

path_to_list('impr(1) -> click(2) -> hva(3)')

In [ ]:
user_path['path_list'] = user_path['path'].apply(path_to_list)

In [ ]:
user_path['length'] = user_path['path_list'].apply(len)

user_path.sort_values(by='length', inplace=True)

user_path.reset_index(inplace=True)

user_path

In [ ]:
all_path = list(user_path['path_list'])

In [ ]:
len(all_path[-1])

In [ ]:
json.dump(all_path, open('/home/jubauser1/jzou/dcm_account7252/user_path/all_path_added_brand_nonbrand.json', 'w'))

In [ ]:
all_path = json.load(open('/home/jubauser1/jzou/dcm_account7252/user_path/all_path.json', 'r'))

In [ ]:
path_permutation = list()
x = 0
for pth in all_path:
    n = len(pth)
    i = 0
    for j in range(len(path_permutation)):
        if pth[i] == path_permutation[j]:
            i += 1
            if i == n: break
    path_permutation.extend(pth[i:n])
    x += 1
    logging.info('%d: %d'%(x, len(path_permutation)))

In [ ]:
set(path_permutation)

In [ ]:
json.dump(path_permutation, open('/home/jubauser1/jzou/dcm_account7252/user_path/path_permutation_added_brand_nonbrand.json', 'w'))

In [ ]:
path_permutation = json.load(open('/home/jubauser1/jzou/dcm_account7252/user_path/path_permutation_added_brand_nonbrand.json', 'r'))

In [ ]:
df = pd.DataFrame([[1,0,2,0, 'asdf'], [1,1,1,0, 'dfs']])
df

df.replace(0, np.nan, inplace=True)

df.notnull().astype('int')

In [ ]:
def path_str_permutaion_counts(s, permutation):
    '''count each activity in order of path_permutaion
    Args:
    s: str, eg. impr(1) -> click(2) -> hva(3)
    permutation: list, eg. [impr, click, impr, hva, transaction]
    
    Retruns,
    list, [1, 2, 0, 3, 0]
    '''
    l = s.split(' -> ')
    i = 0
    result = [0]*len(permutation)
    for x in l:
        act, count = extract_activity_count(x)
        while permutation[i] != act:
            i += 1
        result[i] = count
        i += 1
    return result
        
path_str_permutaion_counts('impr(1) -> click(2) -> hva(3)', ['click', 'impr', 'click', 'impr', 'hva', 'transaction'])

In [ ]:
import math
math.pow(10, 3)

In [ ]:
sample_index = list()
for i in range(7):
    sample_index += range(int(math.pow(10, i)), int(math.pow(10, i)+100))

In [ ]:
count_by_permutation = list()
for index in user_path.index:
#for index in sample_index:
    logging.info(index)
    pth = user_path.loc[index, 'path']
    result = path_str_permutaion_counts(pth, path_permutation)
    result.append(user_path.loc[index, 'user_id'])
    count_by_permutation.append(result)

In [ ]:
sys.getsizeof(count_by_permutation)

In [ ]:
len(count_by_permutation)

In [ ]:
len(count_by_permutation[0])

In [ ]:
json.dump(count_by_permutation, open('/home/jubauser1/jzou/dcm_account7252/user_path/count_by_permutation_added_brand_nonbrand.json', 'w'))

In [ ]:
count_by_permutation = json.load(open('/home/jubauser1/jzou/dcm_account7252/user_path/count_by_permutation.json', 'r'))

In [ ]:
len(count_by_permutation)

In [ ]:
path_permutation.append('user_id')
count_by_permutation.insert(0, path_permutation)

In [ ]:
count_by_permutation_df = pd.DataFrame(count_by_permutation, dtype=int)

In [ ]:
count_by_permutation_df.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/count_by_permutation_sample.csv', index=False)

In [ ]:
count_by_permutation_df.replace(0, np.nan, inplace=True)

count_by_permutation_df.notnull().astype(int).to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/count_by_permutation_binary_sample.csv', index=False)

In [ ]:
count_by_permutation_df

# 3 step path

In [ ]:
merged

In [ ]:
def path_list_to_str_no_count(path):
    '''
    Args: 
        path: list, ['a', 'a', 'b', 'c', 'c', 'c']
    Returns:
        str, 'a(2) -> b(1) -> c(3)'
    '''
    #if 'transaction' in path:
    #    path = path[: path.index('transaction')+1]
    curr = path[0]
    result = list()
    for x in path:
        if x == curr:
            continue
        else:
            result.append(curr)
            curr = x
    result.append(curr)
    return result

path_list_to_str_no_count(['a', 'a', 'others', 'b', 'others', 'transaction', 'c', 'c',  'c', 'others'])

In [ ]:
import logging
logging.basicConfig(filename='path.log', 
                    level=logging.INFO, 
                    format='%(asctime)s %(message)s')

In [ ]:
l = ['a', 'b']
l.count('c')

In [ ]:
acts = set(merged['activity_cat'])

In [ ]:
acts

In [ ]:
medias = ['branded_click: DART Search : Google',
         'branded_click: DART Search : MSN',
         'click: Criteo',
         'click: DART Search : Google',
         'click: DART Search : MSN',
         'click: Facebook',
         'click: QuantCast',
         'click: Rakuten Marketing Index',
         'impr: Criteo',
         'impr: QuantCast',
         'impr: Rakuten Marketing Index',
         'nonbranded_click: DART Search : Google',
         'nonbranded_click: DART Search : MSN',
]

In [ ]:
def extract_2_steps_sequence(l):
    '''extract all combinations in order, eg. ['a', 'b', 'c'] 
    will return ['a_to_b', 'a_to_c', 'b_to_c'], 'c_to_b' won't 
    be in result, since there was no 'b' after 'c'
    Args:
        l, list of string
    Returns:
        result, list of string
    '''
    if len(l) == 1:
        return ['%s -> nothing'%l[0]]
    elif len(l) == 0:
        return ['no_media']
    else:
        d = {l[0]: set()}
        for i in range(1, len(l)):
            for v in d.values():
                v.add(l[i])
            if l[i] not in d:
                d[l[i]] = set()
        result = list()
        for k in d.keys():
            for v in d[k]:
                result.append('%s -> %s'%(k, v))
        return result
    
extract_2_steps_sequence([])

In [ ]:
def count_hva_transaction(l):
    '''count how many hva and transaction in list l'''
    d = dict()
    for x in {'others', 'HVA1', 'HVA2', 'HVA3', 'transaction'}:
        d[x] = l.count(x)
    return d

count_hva_transaction(['a', 'HVA1', 'HVA1', 'HVA2'])

In [ ]:
merged[merged['User ID'] == '']

In [ ]:
merged_groups_by_id = merged.groupby('User ID')

user_paths = list()
for user, group in merged_groups_by_id:
    try:
        path_list = list(group['activity_cat'])
        #path_list = [x for x in path_list if x != 'others']
        path_list = [x for x in path_list if x not in {'others', 'HVA1', 'HVA2', 'HVA3', 'transaction'}]
        if len(path_list) > 0:
            path = path_list_to_str_no_count(path_list)
            logging.info('%s >>>> %s'%(user, path))
            user_path_dict[user] = path
    except Exception as e:
        print(user, e)

In [ ]:
user_path = pd.DataFrame({'user_id': list(user_path_dict.keys()), 'path': list(user_path_dict.values())})

In [ ]:
user_path.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/user_path_no_count.csv', index=False)

In [ ]:
user_path = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/user_path_no_count.csv')

In [ ]:
user_path

In [ ]:
sample_user = ['AMsySZYqZ62isOj4ct2oaN7cxzGm',
    'AMsySZbvt4-cUPcPkym8okp_IvkL',
    'AMsySZaecrQgZiLlYDllkIP-qrsI',
    'AMsySZZx2SL_OWuDkcb1kLFoxZSf',
    'AMsySZYPo16_Thv1pOtns8PChLud',
    'AMsySZanHUWC9p2FfsUw2bjrJK0u',
    'AMsySZYa2OIWa7yfPWOseKc2a8K0',
    'AMsySZaeArP_muso7MrpJOmZcBng',
    'AMsySZY3yrWNDMK_M7rgKIqdjBYS',
    'AMsySZa0hlZABPXiP6U2inxD3M-l']

path_by_user = pd.DataFrame()
for user_id in sample_user:
    l = list(merged[merged['User ID'] == user_id]['activity_cat'])
    media_flag = True
    result = dict()
    conversions = {'others', 'HVA1', 'HVA2', 'HVA3', 'transaction'}
    for i in range((len(l))):
        if (l[i] in conversions or i == len(l)-1) and media_flag:
            media_flag = False
            _i = i+1 if i==len(l)-1 else i
            sequences = extract_2_steps_sequence([x for x in l[:_i] if x not in conversions])
            counts = count_hva_transaction(l[i:])
            #counts['user_id'] = user_id
            for s in sequences:
                if s not in result:
                    result[s] = counts
        if l[i] not in conversions and not media_flag:
            media_flag = True

    df = pd.DataFrame(result).T
    df['user_id'] = user_id
    path_by_user = path_by_user.append(df)

In [ ]:
merged.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/merged.csv', index=False)

In [ ]:
from collections import Counter
def result_cum(d1, d2):
    for k, v in d2.items():
        if k in d1:
            d1[k] = dict(Counter(d1[k]) + Counter(d2[k]))
        else:
            d1[k] = d2[k]

In [ ]:
path_by_user = pd.DataFrame()
cum_result = dict()
for user_id in sample_user:
    l = list(merged[merged['User ID'] == user_id]['activity_cat'])
    media_flag = True
    result = dict()
    conversions = {'others', 'HVA1', 'HVA2', 'HVA3', 'transaction'}
    for i in range((len(l))):
        if (l[i] in conversions or i == len(l)-1) and media_flag:
            media_flag = False
            _i = i+1 if i==len(l)-1 else i
            sequences = extract_2_steps_sequence([x for x in l[:_i] if x not in conversions])
            counts = count_hva_transaction(l[i:])
            counts['user_id'] = 1
            for s in sequences:
                if s not in result:
                    result[s] = counts
        if l[i] not in conversions and not media_flag:
            media_flag = True
    
    result_cum(cum_result, result)

In [ ]:
path_by_user

In [ ]:
groupby_path = path_by_user.groupby(path_by_user.index)
temp = groupby_path[list(conversions)].sum()

temp['users'] = groupby_path['user_id'].count()

In [ ]:
temp

In [ ]:
df = pd.DataFrame(cum_result).T
df

In [ ]:
temp.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/three_step_path.csv')

In [ ]:
temp = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/three_step_path_dict.csv')

In [ ]:
temp['first_step'] = temp['Unnamed: 0'].apply(lambda x: x.split(' -> ')[0])

In [ ]:
temp['second_step'] = temp['Unnamed: 0'].apply(lambda x: x.split(' -> ')[1] if len(x.split(' -> '))>1 else x)

In [ ]:
temp.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/three_step_path_dict_split.csv')

# start point of path

In [4]:
merged = pd.read_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/merged_with_cost_and_group_no_others_2017-08-15.csv')

In [5]:
merged

,Event Time,User ID,activity_cat,cost,revenue,group
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,click: QuantCast,NaN,NaN,3
1,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,click: QuantCast,NaN,NaN,4
2,1490969529196517,AMsySZbCkYLEzuC2-R0IJyWJh51l,click: QuantCast,NaN,NaN,4
...,...,...,...,...,...,...
9878709,1504497596191373,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0.0,3
9878710,1504497596212905,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0.0,3
9878711,1504497599912465,AMsySZaAToWbFircoFIR0y8Ipxhd,HVA2,NaN,0.0,3


In [6]:
merged['revenue'].sum()

687728.31000000006

In [7]:
merged['cost'].sum()

135757.00908132232

In [8]:
first_point = merged.drop_duplicates('User ID', keep='first')

In [23]:
path_start_point = pd.DataFrame(
    first_point.groupby('activity_cat')['group'].value_counts().rename('user_id')
).reset_index().pivot(index='activity_cat', columns='group')

In [27]:
path_start_point

user_id                       
group                                        1       2       3      4
activity_cat                                                         
HVA1                                      13.0   113.0   131.0    NaN
HVA2                                     241.0  2167.0  6247.0    NaN
HVA3                                       8.0     5.0     2.0    NaN
...                                        ...     ...     ...    ...
nonbranded_click: DART Search : Google   196.0  1028.0  8345.0  914.0
nonbranded_click: DART Search : MSN       43.0   240.0  1568.0  518.0
transaction                                3.0     NaN     NaN    NaN

[15 rows x 4 columns]

In [34]:
path_start_point['user_id'][[1,2,3,4]].sum(axis=1)

activity_cat
HVA1                                        257.0
HVA2                                       8655.0
HVA3                                         15.0
                                           ...   
nonbranded_click: DART Search : Google    10483.0
nonbranded_click: DART Search : MSN        2369.0
transaction                                   3.0
Length: 15, dtype: float64

In [35]:
path_start_point['user_id']['total_user_id'] = path_start_point['user_id'][[1,2,3,4]].sum(axis=1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
path_start_point.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/path_start_point.csv')

# 4 step path

In [2]:
import itertools

In [3]:
merged = pd.read_csv(
    '/home/jubauser1/jzou/dcm_account7252/user_path/merged_with_cost_and_group_no_others_2017-08-15.csv')

In [4]:
merged

,Event Time,User ID,activity_cat,cost,revenue,group
0,1490902299891058,AMsySZaTfbByUHA4eFC-oaXSEcel,click: QuantCast,NaN,NaN,3
1,1490969409208755,AMsySZbyH3iw4NUZ-TZoCia-_fQ7,click: QuantCast,NaN,NaN,4
2,1490969529196517,AMsySZbCkYLEzuC2-R0IJyWJh51l,click: QuantCast,NaN,NaN,4
...,...,...,...,...,...,...
9878709,1504497596191373,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0.0,3
9878710,1504497596212905,AMsySZYqOIi_FMhRk1cWNHS1gB-O,HVA2,NaN,0.0,3
9878711,1504497599912465,AMsySZaAToWbFircoFIR0y8Ipxhd,HVA2,NaN,0.0,3


In [108]:
def path_list_count_unique(path, cost):
    '''
    Args: 
        path: list, ['a', 'a', 'b', 'c', 'c', 'c']
    Returns:
        dict, {'path_str': 'a -> b -> c'
               'path': [a, b, c],
               'counts': [2, 1, 3]}
    '''
    curr = path[0]
    count = 0
    step_cost = 0
    result = {'path': [], 'counts': [], 'cost': []}
    for i in range(len(path)):
        if path[i] == curr:
            count += 1
            step_cost += cost[i]
        else:
            result['path'].append(curr)
            result['counts'].append(count)
            result['cost'].append(step_cost)
            curr = path[i]
            count = 1
            step_cost = cost[i]
    result['path'].append(curr)
    result['counts'].append(count)
    result['cost'].append(step_cost)
    result['path_str'] = ' -> '.join(result['path'])
    return result

path_list_count_unique(['a', 'a', 'others', 'b', 'others', 'c', 'c',  'transaction','c', 'others'], 
                       list(range(10)))

{'cost': [1, 2, 3, 4, 11, 7, 8, 9],
 'counts': [2, 1, 1, 1, 2, 1, 1, 1],
 'path': ['a', 'others', 'b', 'others', 'c', 'transaction', 'c', 'others'],
 'path_str': 'a -> others -> b -> others -> c -> transaction -> c -> others'}

In [9]:
sys.getsizeof(merged)

1848059626

In [75]:
l = list(merged[merged['User ID'] == 'AMsySZZmsJbSxZxBuzi9HpSxl6cu']['activity_cat'])

In [77]:
l2 = path_list_count_unique(l)

In [78]:
l2

{'counts': [1, 1, 2, 5, 2, 4, 2, 50, 3, 3, 2, 6, 1, 1],
 'path': ['impr: QuantCast',
  'click: QuantCast',
  'HVA2',
  'impr: QuantCast',
  'HVA2',
  'impr: QuantCast',
  'HVA2',
  'impr: QuantCast',
  'HVA2',
  'HVA3',
  'HVA2',
  'HVA3',
  'HVA1',
  'HVA3'],
 'path_str': 'impr: QuantCast -> click: QuantCast -> HVA2 -> impr: QuantCast -> HVA2 -> impr: QuantCast -> HVA2 -> impr: QuantCast -> HVA2 -> HVA3 -> HVA2 -> HVA3 -> HVA1 -> HVA3'}

In [90]:
is_sub_list(l2['path'], ('impr: QuantCast', 'click: QuantCast'))

True

In [ ]:
path_and_cost = list()
for user_id, group in merged.groupby('User ID'):
    if 'transaction' in set(group['activity_cat']):
        index_of_1st_transaction = group[group['activity_cat'] == 'transaction'].index[0]
        group = group.loc[:index_of_1st_transaction]
    group = group[group['activity_cat'].isin(steps)]
    if len(group) == 0: continue
    d = path_list_count_unique(list(group['activity_cat']), list(group['cost'].fillna(0)))
    d['user_id'] = user_id
    path_and_cost.append(d)

In [ ]:
path_and_cost_by_user = pd.DataFrame(path_and_cost)

In [5]:
path_and_cost_by_user = pd.read_csv('/home/jubauser1/jzou/dcm_account7252/user_path/path_cost_by_user.csv')

In [13]:
path_and_cost_by_user = path_and_cost_by_user.merge(merged[['User ID', 'group']].drop_duplicates('User ID'), left_on='user_id', right_on='User ID', how='left')

path_and_cost_by_user[path_and_cost_by_user['group'] == 1].drop_duplicates('User ID')

,cost,counts,path,path_str,user_id,User ID,group
548,[1.069672131147541],[1],[click: DART Search : Google],click: DART Search : Google,AMsySZY-3FnuknZAk_Iuw_o7vZXF,AMsySZY-3FnuknZAk_Iuw_o7vZXF,1
643,"[0.2843492353743657, 0.0, 0.09033840775770098,...","[110, 1, 45, 1, 155, 1, 48, 1]","[impr: Criteo, click: Criteo, impr: Criteo, br...",impr: Criteo -> click: Criteo -> impr: Criteo ...,AMsySZY-3texhhoZd8Dov2qqQvpN,AMsySZY-3texhhoZd8Dov2qqQvpN,1
1708,"[0.14987960190620492, 2.6852132701421803, 0.05...","[75, 3, 1, 4, 2]","[impr: Criteo, click: DART Search : Google, br...",impr: Criteo -> click: DART Search : Google ->...,AMsySZY-BLK48LeaEQ2Na54gg8xM,AMsySZY-BLK48LeaEQ2Na54gg8xM,1
...,...,...,...,...,...,...,...
2366376,"[0.19341085019242615, 0.335, 0.004181313598519...","[78, 1, 2, 1]","[impr: Criteo, branded_click: DART Search : Go...",impr: Criteo -> branded_click: DART Search : G...,AMsySZbzvNKPEjY24T4ae9YRu1u9,AMsySZbzvNKPEjY24T4ae9YRu1u9,1
2366421,[0.9483333333333334],[1],[branded_click: DART Search : Google],branded_click: DART Search : Google,AMsySZbzvjSvjfHe5hdYH9wNZYxP,AMsySZbzvjSvjfHe5hdYH9wNZYxP,1
2366991,"[0.9001273885350318, 0.3205882352941177]","[1, 1]","[click: DART Search : Google, branded_click: D...",click: DART Search : Google -> branded_click: ...,AMsySZbzzXQ_uBd53piNXo4vEbDm,AMsySZbzzXQ_uBd53piNXo4vEbDm,1


In [15]:
del path_and_cost_by_user['User ID']

In [7]:
path_and_cost_by_user['cost'] = path_and_cost_by_user['cost'].apply(eval)
path_and_cost_by_user['counts'] = path_and_cost_by_user['counts'].apply(eval)
path_and_cost_by_user['path'] = path_and_cost_by_user['path'].apply(eval)

In [8]:
def is_sub_list(long_list, short_list):
    '''determine if short_list is sublist of long_list
    eg. [1, 3, 5] is sublist of [0, 1, 2, 3, 4, 5]
    Args:
        long_list: list of str
        short_list: iterable obj, tuple or list
    Return:
        list of index
    '''
    x = 0
    result = list()
    for i in short_list:
        try:
            x = long_list.index(i, x)+1
            result.append(x-1)
        except ValueError:
            return []
    return result

In [9]:
short_list = [1, 6, 5, ]
long_list = [0, 1, 2, 3, 1, 6, 4, 5]

is_sub_list(long_list, short_list)

[1, 5, 7]

In [10]:
[long_list[i] for i in [1,5,7]]

[1, 6, 5]

In [71]:
steps = {'branded_click: DART Search : Google',
         'branded_click: DART Search : MSN',
         'click: Criteo',
         'click: DART Search : Google',
         'click: DART Search : MSN',
         'click: Facebook',
         'click: QuantCast',
         'impr: Criteo',
         'impr: QuantCast',
         'nonbranded_click: DART Search : Google',
         'nonbranded_click: DART Search : MSN',}

path_lib = dict()
path_lib[1] = list(itertools.permutations(steps, r=1))
path_lib[2] = list(itertools.permutations(steps, r=2))
path_lib[3] = list(itertools.permutations(steps, r=3))
path_lib[4] = list(itertools.permutations(steps, r=4))

path_lib[4][1]

('click: Facebook',
 'click: QuantCast',
 'click: DART Search : MSN',
 'impr: QuantCast')

In [18]:
path_and_cost_by_user['path'].apply(lambda x: len(set(x)))

7

In [23]:
path_and_cost_by_user[path_and_cost_by_user['group'] == 1]

,cost,counts,path,path_str,user_id,group
548,[1.069672131147541],[1],[click: DART Search : Google],click: DART Search : Google,AMsySZY-3FnuknZAk_Iuw_o7vZXF,1
643,"[0.2843492353743657, 0.0, 0.09033840775770098,...","[110, 1, 45, 1, 155, 1, 48, 1]","[impr: Criteo, click: Criteo, impr: Criteo, br...",impr: Criteo -> click: Criteo -> impr: Criteo ...,AMsySZY-3texhhoZd8Dov2qqQvpN,1
1708,"[0.14987960190620492, 2.6852132701421803, 0.05...","[75, 3, 1, 4, 2]","[impr: Criteo, click: DART Search : Google, br...",impr: Criteo -> click: DART Search : Google ->...,AMsySZY-BLK48LeaEQ2Na54gg8xM,1
...,...,...,...,...,...,...
2366376,"[0.19341085019242615, 0.335, 0.004181313598519...","[78, 1, 2, 1]","[impr: Criteo, branded_click: DART Search : Go...",impr: Criteo -> branded_click: DART Search : G...,AMsySZbzvNKPEjY24T4ae9YRu1u9,1
2366421,[0.9483333333333334],[1],[branded_click: DART Search : Google],branded_click: DART Search : Google,AMsySZbzvjSvjfHe5hdYH9wNZYxP,1
2366991,"[0.9001273885350318, 0.3205882352941177]","[1, 1]","[click: DART Search : Google, branded_click: D...",click: DART Search : Google -> branded_click: ...,AMsySZbzzXQ_uBd53piNXo4vEbDm,1


In [27]:
list(itertools.permutations([1,2, 3], r=3))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [29]:
is_sub_list([1,2,3], (1, 3, 2))

[]

In [36]:
four_step_path = dict()

In [44]:
index = 1708
_path = path_and_cost_by_user.loc[index, 'path']
_cost = path_and_cost_by_user.loc[index, 'cost']
_counts = path_and_cost_by_user.loc[index, 'counts'] 
_user_id = path_and_cost_by_user.loc[index, 'user_id'] 

len_of_path = len(set(_path))
if len_of_path <= 4:
    possible_path = path_lib[len_of_path]
else:
    possible_path = path_lib[4]

for _p_path in possible_path:
    list_of_index = is_sub_list(_path, _p_path)
    if len(list_of_index) > 0:
        d = dict()
        d['user_id'] = _user_id
        d['counts'] = [_counts[i] for i in list_of_index]
        d['cost'] = sum([_cost[i] for i in list_of_index])
        if _p_path in four_step_path:
            four_step_path.append(d)
        else:
            four_step_path[_p_path] = [d]         

In [49]:
four_step_path.keys()

dict_keys([('impr: Criteo', 'click: Criteo', 'branded_click: DART Search : Google'), ('click: Criteo', 'impr: Criteo', 'branded_click: DART Search : Google'), ('click: Criteo', 'branded_click: DART Search : Google', 'impr: Criteo'), ('impr: Criteo', 'click: DART Search : Google', 'branded_click: DART Search : Google', 'click: Criteo'), ('click: DART Search : Google', 'branded_click: DART Search : Google', 'impr: Criteo', 'click: Criteo')])

In [41]:
_path

['impr: Criteo',
 'click: Criteo',
 'impr: Criteo',
 'branded_click: DART Search : Google',
 'impr: Criteo',
 'branded_click: DART Search : Google',
 'impr: Criteo',
 'branded_click: DART Search : Google']

In [42]:
_counts

[110, 1, 45, 1, 155, 1, 48, 1]

In [43]:
_cost

[0.2843492353743657,
 0.0,
 0.09033840775770098,
 0.6633333333333333,
 0.4603856769949713,
 0.18583333333333327,
 0.13977578626061257,
 0.14473684210526316]

In [40]:
four_step_path

{('click: Criteo',
  'branded_click: DART Search : Google',
  'impr: Criteo'): [{'cost': 1.1237190103283046,
   'counts': [1, 1, 155],
   'user_id': 'AMsySZY-3texhhoZd8Dov2qqQvpN'}],
 ('click: Criteo',
  'impr: Criteo',
  'branded_click: DART Search : Google'): [{'cost': 0.7536717410910343,
   'counts': [1, 45, 1],
   'user_id': 'AMsySZY-3texhhoZd8Dov2qqQvpN'}],
 ('impr: Criteo',
  'click: Criteo',
  'branded_click: DART Search : Google'): [{'cost': 0.9476825687076991,
   'counts': [110, 1, 1],
   'user_id': 'AMsySZY-3texhhoZd8Dov2qqQvpN'}]}

In [38]:
sum([1,23,4])

28

In [51]:
import logging
logging.basicConfig(filename='/home/jubauser1/jzou/dcm_account7252/user_path/scripts/path.log', 
                    level=logging.INFO, 
                    format='%(asctime)s %(message)s')

In [102]:
four_step_path_no_visit = dict()
for index in path_and_cost_by_user[path_and_cost_by_user['group'] == 4].index:
    _path = path_and_cost_by_user.loc[index, 'path']
    _cost = path_and_cost_by_user.loc[index, 'cost']
    _counts = path_and_cost_by_user.loc[index, 'counts'] 
    _user_id = path_and_cost_by_user.loc[index, 'user_id'] 
    logging.info(index)
    len_of_path = len(set(_path))
    if len_of_path <= 4:
        possible_path = path_lib[len_of_path]
    else:
        possible_path = path_lib[4]

    for _p_path in possible_path:
        list_of_index = is_sub_list(_path, _p_path)
        if len(list_of_index) > 0:
            d = dict()
            d['user_id'] = _user_id
            d['counts'] = [_counts[i] for i in list_of_index]
            d['cost'] = sum([_cost[i] for i in list_of_index])
            if str(_p_path) in four_step_path_no_visit:
                four_step_path_no_visit[str(_p_path)].append(d)
            else:
                four_step_path_no_visit[str(_p_path)] = [d]  

In [95]:
import json
json.dump(four_step_path_buyers, open('/home/jubauser1/jzou/dcm_account7252/user_path/four_step_path_buyers.json', 'w'))

In [98]:
json.dump(four_step_path_high_engagers, 
          open('/home/jubauser1/jzou/dcm_account7252/user_path/four_step_path_high_engagers.json', 'w'))

In [101]:
json.dump(four_step_path_low_engagers, 
          open('/home/jubauser1/jzou/dcm_account7252/user_path/four_step_path_low_engagers.json', 'w'))

In [104]:
json.dump(four_step_path_no_visit, 
          open('/home/jubauser1/jzou/dcm_account7252/user_path/four_step_path_no_visit.json', 'w'))

In [103]:
len(four_step_path_buyers), len(four_step_path_high_engagers), len(four_step_path_low_engagers), len(four_step_path_no_visit)

(873, 2237, 946, 134)

In [180]:
path_by_group = {'buyers': four_step_path_buyers,
               'high_engagers': four_step_path_high_engagers,
               'low_engagers': four_step_path_low_engagers,
               'no_visit': four_step_path_no_visit}
writer = pd.ExcelWriter('/home/jubauser1/jzou/dcm_account7252/user_path/frequency_of_four_step_path_by_groups.xlsx')

for _p, _length in path_by_group.items():
    path_length = dict()
    for k, v in _length.items():
        path_length[k] = len(v)

    temp = pd.Series(path_length).sort_values(ascending=False)

    temp.name=_p

    temp2 = pd.DataFrame(temp)

    temp2['path'] = temp2.index

    temp2['path'] = temp2['path'].apply(lambda x: eval(x))

    temp2['step1'] = temp2['path'].apply(lambda x: x[0] if len(x) >= 1 else '')
    temp2['step2'] = temp2['path'].apply(lambda x: x[1] if len(x) >= 2 else '')
    temp2['step3'] = temp2['path'].apply(lambda x: x[2] if len(x) >= 3 else '')
    temp2['step4'] = temp2['path'].apply(lambda x: x[3] if len(x) >= 4 else '')

    del temp2['path']
    temp2.to_excel(writer, sheet_name=_p, index=False)
    
    writer2 = pd.ExcelWriter('/home/jubauser1/jzou/dcm_account7252/user_path/top_10_%s_paths.xlsx'%_p)
    top10_paths = list(temp2.index)[:10]
    for i in range(10):
        _path = top10_paths[i]
        temp3 = pd.DataFrame(_length[_path])
        temp3 = temp3.join(pd.DataFrame(list(temp3['counts']), columns=eval(_path)))
        del temp3['counts']
        temp3.to_excel(writer2, sheet_name='path%d'%i, index=False)
    writer2.save()
writer.save()

In [182]:
top10_paths[0]

"('impr: QuantCast',)"

In [183]:
temp3 = pd.DataFrame(_length[top10_paths[0]])

In [184]:
temp3

,cost,counts,user_id
0,0.009694,[3],AMsySZY--0PH7CoXjhqmBxijl-Ia
1,0.000824,[1],AMsySZY--0olr34AoV4fpH5jQMxU
2,0.003829,[1],AMsySZY--0tGbw00Jq7wk4Y4n8RB
...,...,...,...
2130593,0.003825,[1],AMsySZbzzzPCrgcK7aLHmifIekd7
2130594,0.000669,[1],AMsySZbzzzdemn_t_OqL6zxBElbD
2130595,0.011128,[1],AMsySZbzzzgrka8IJ3oxBumYjVZ4


In [185]:
temp3 = temp3.join(pd.DataFrame(list(temp3['counts']), columns=eval(top10_paths[0])))

In [187]:
del temp3['counts']

In [188]:
temp3.to_csv('/home/jubauser1/jzou/dcm_account7252/user_path/top_1_no_visit_path.csv', index=False)